<a href="https://colab.research.google.com/github/WilliamPoe/CSCI-290/blob/main/notebooks/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math

In [12]:
def cat_entropy(df, target, attribs):
  entropy = 0
  lowest_entropy = float('inf')
  split_point = 0
  lowest_quant_entropy = float('inf')
  quant_attrib = ''
  overall = len( df )
  ## Remove the [2:] ##
  for attrib in attribs[2:]:
    vals = df[attrib].unique()
    # Quantitative Entropy Calc
    if df[attrib].dtype in ['float64', 'int64']:
      if len( df[attrib].unique() ) > 10:
        vals.sort()
        overall = len( df )
        entropies = []
        for val in vals:
          entropy = 0
          left = df[ df[attrib] <= val ][ [attrib,target] ]
          props = left[ target ].value_counts( normalize = True )
          weight = len( left ) / overall
          for prop in props.array:
            entropy = entropy - weight*prop*math.log2( prop )
          right = df[ df[attrib] > val ][ [attrib,target] ]
          props = right[ target ].value_counts( normalize = True )
          weight = len( right ) / overall
          for prop in props.array:
            if prop > 0:
              entropy = entropy - weight*prop*math.log2( prop )
            entropies.append(entropy)
        index = pd.Series( entropies ).idxmin()
        # Check to see if there is a new lowest Entropy
        if entropies[index] < lowest_quant_entropy:
          # Lowest entropy
          lowest_quant_entropy = entropies[index]
          # Split point with lowest entropy
          split_point = vals[index]
          # Attribute that has the split point
          quant_attrib = attrib
    # Categorical Entropy Calc
    else:
      for val in vals:
        subset_size = len(df[ df[attrib] == val ])
        weight = subset_size / overall
        props = df[ df[attrib] == val ][target].value_counts( normalize=True )
        for p in props.array:
          entropy =  entropy - weight*(p*math.log2(p))
      if lowest_entropy > entropy:
        lowest_entropy = entropy
        lowest_attrib = attrib
  # Compares the lowest categorical Entropy with the lowest quantitative Entropy
  if lowest_entropy < lowest_quant_entropy:
    # If categorical entropy is still lowest
    return lowest_attrib, lowest_entropy
  else:
    # If quantitative entropy is lowest
    lowest_entropy = lowest_quant_entropy
    lowest_attrib = quant_attrib
    return lowest_attrib, split_point, lowest_entropy


In [3]:
# possible check what the values are for that attribute and then call a function that does
# categorical or quantitative if it has more than 10 values
# categorical name and entropy
# quantitative name and splitting point

In [4]:
def cat_gini_impurity(df, target, attribs):
  gini = 0
  lowest_gini = float('inf')
  overall = len( df )
  ## Remove the [2:] ##
  for attrib in attribs[2:]:
    vals = df[attrib].unique()
    for val in vals:
      subset_size = len(df[ df[attrib] == val ])
      weight = subset_size / overall
      props = df[ df[attrib] == val ][target].value_counts( normalize=True )
      for p in props.array:
        gini =  weight*(1-(p**2))
    if lowest_gini > gini:
      lowest_gini = gini
      lowest_attrib = attrib
  return lowest_attrib, lowest_gini

In [5]:
def main ():
  df = pd.read_csv("https://github.com/WilliamPoe/CSCI-290/raw/main/Data/ad_click_dataset.csv")
  attribs = []
  print(df.columns)
  target = str(input("What is your target column?: "))
  for col in df.columns:
    if col != target:
      attribs.append(col)
  measure = input("Would you like to use: \n1. Entropy \n2. Gini impurity \nSelection:")
  if measure == '1':
    print(cat_entropy(df, target, attribs))
  else:
    print(cat_gini_impurity(df, target, attribs))

In [10]:
main()

Index(['id', 'full_name', 'age', 'gender', 'device_type', 'ad_position',
       'browsing_history', 'time_of_day', 'click'],
      dtype='object')
What is your target column?: click
Would you like to use: 
1. Entropy 
2. Gini impurity 
Selection:1
It worked!
('age', 18.0, 0.21468933295874262)


In [11]:
main()

Index(['id', 'full_name', 'age', 'gender', 'device_type', 'ad_position',
       'browsing_history', 'time_of_day', 'click'],
      dtype='object')
What is your target column?: click
Would you like to use: 
1. Entropy 
2. Gini impurity 
Selection:2
('age', 0.005131578947368422)
